In [1]:
using Pkg
Pkg.status()
using SymPy
using BenchmarkTools, LinearAlgebra, LazyArrays, BandedMatrices

      Status `~/.julia/environments/v1.7/Project.toml`
  [aae01518] BandedMatrices v0.17.0
  [6e4b80f9] BenchmarkTools v1.3.1
  [8e7c35d0] BlockArrays v0.16.18
  [ce91de38] ColorBitstring v0.1.1
  [7073ff75] IJulia v1.23.3
  [5078a376] LazyArrays v0.22.10
  [a3b82374] MatrixFactorizations v0.9.1
  [91a5bcdd] Plots v1.29.0
  [f27b6e38] Polynomials v3.1.4
  [438e738f] PyCall v1.93.1
  [d330b81b] PyPlot v2.10.0
  [295af30f] Revise v3.3.3
  [f8ebbe35] SemiseparableMatrices v0.3.3
  [3cc68bcd] SetRounding v0.2.1
  [24249f21] SymPy v1.1.6
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random
  [8dfed614] Test


In [3]:
function ppl(context)
    println(sympy.latex(context))
end

ppl (generic function with 1 method)

In [15]:
xx0 = symbols("x_(1:6)", real=True)
yy0 = symbols("y_(1:6)", real=True)
xs = [i for i in xx0]
ys = [i for i in yy0]
xf = rand(6)
yf = rand(6);

In [16]:
Ss = triu(xs * ys', 1) + tril(ys * xs')

5×5 Matrix{Sym}:
 x₁⋅y₁  x₁⋅y₂  x₁⋅y₃  x₁⋅y₄  x₁⋅y₅
 x₁⋅y₂  x₂⋅y₂  x₂⋅y₃  x₂⋅y₄  x₂⋅y₅
 x₁⋅y₃  x₂⋅y₃  x₃⋅y₃  x₃⋅y₄  x₃⋅y₅
 x₁⋅y₄  x₂⋅y₄  x₃⋅y₄  x₄⋅y₄  x₄⋅y₅
 x₁⋅y₅  x₂⋅y₅  x₃⋅y₅  x₄⋅y₅  x₅⋅y₅

In [20]:
Sf = triu(xf * yf', 1) + tril(yf * xf')
rank(Sf)

6

In [21]:
inv(Sf)

6×6 Matrix{Float64}:
 128.155        -15.6791        2.14982e-15  …   6.67994e-15  -6.55774e-17
 -15.6791         1.29027       1.32858         -6.91199e-16   2.36326e-18
   1.19748e-15    1.32858       0.333956        -5.92087e-16  -2.16045e-17
  -3.56515e-15    2.69116e-16  -1.52385         -5.82871      -4.46382e-17
   5.25333e-15   -5.19112e-16  -2.67878e-16      3.8143        3.58752
  -1.70601e-16    2.40604e-17   2.84286e-18  …   3.58752      -0.174701

In [22]:
Tridiagonal(inv(Sf))

6×6 Tridiagonal{Float64, Vector{Float64}}:
 128.155   -15.6791     ⋅          ⋅         ⋅         ⋅ 
 -15.6791    1.29027   1.32858     ⋅         ⋅         ⋅ 
    ⋅        1.32858   0.333956  -1.52385    ⋅         ⋅ 
    ⋅         ⋅       -1.52385    5.00978  -5.82871    ⋅ 
    ⋅         ⋅         ⋅        -5.82871   3.8143    3.58752
    ⋅         ⋅         ⋅          ⋅        3.58752  -0.174701

In [23]:
isposdef(Tridiagonal(inv(Sf)))

false

In [47]:
n = 6
A = rand(n,n)
T = A * A'
isposdef(T)
C = cholesky(T)
L = C.L
U = C.U
L * L' ≈ T
U' * U ≈ T

true

In [45]:
include("backsub.jl")

true

In [44]:
# LU(T)



true